In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

API_KEY = os.getenv('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = API_KEY

In [3]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.vectorstores.elastic_vector_search import ElasticVectorSearch
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.vectorstores.faiss import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import OpenAI
import pickle

In [15]:
def ask_no_history(qa, query):
    return qa({"question": query, "chat_history": []})

def ask_with_history_and_append(qa, query, history):
    res = qa({"question": query, "chat_history": history})
    history.append((query, res["answer"]))
    return res

def pretty_print_res(query, res, include_sources=True):
    print(f"[USER]: {query}")
    print(f"[PenDragon]: {res['answer']}\n")
    print("[PenDragon Sources Consulted]:")
    
    for idx, source in enumerate(res["source_documents"]):
        print(f"--- [{idx+1}] {source.page_content}")

In [4]:
embeddings = OpenAIEmbeddings()

In [6]:
db = None
with open("./db_everything_ultra_processed.pkl", "rb") as f:
    db = pickle.load(f)

In [18]:
qa = ConversationalRetrievalChain.from_llm(
    OpenAI(temperature=0.5),
    db.as_retriever(search_kwargs={"k": 10}),
    return_source_documents=True
)

In [25]:
chat_history = []

In [22]:
query = "Which CS professors are on leave?"
res = ask_with_history_and_append(qa, query, chat_history)
pretty_print_res(query, res)

[USER]: Which CS professors are on leave?
[PenDragon]:  Samuel McCauley, Aaron M. Williams, and Shikha Singh are on leave.

[PenDragon Sources Consulted]:
--- [1] Samuel McCauley, Assistant Professor of Computer Science; on leave 2022-2023 

--- [2] Aaron M. Williams, Associate Professor of Computer Science; on leave Spring 2023 

--- [3] Shikha Singh, Assistant Professor of Computer Science; on leave 2022-2023 

--- [4] Iris Howley, Assistant Professor of Computer Science 

--- [5] Mark Hopkins, Assistant Professor of Computer Science 

--- [6] Bill K. Jannen, Assistant Professor of Computer Science 

--- [7] Kelly A. Shaw, Professor of Computer Science 

--- [8] Stephen N. Freund, Chair and A. Barton Hepburn Professor of Computer Science 

--- [9] James M. Bern, Assistant Professor of Computer Science 

--- [10] Katie A. Keith, Assistant Professor of Computer Science 



In [26]:
chat_history

[]